### Inserting Pictures in the database

In [4]:
import os
import psycopg2
from psycopg2 import sql
import datetime
from config import config

# Set the global variable db_config as the database login credentials
db_config = config()

# Connect to the database
connection = psycopg2.connect(**db_config)

# Specify the folder path and place
folder_path = 'D:\\Misc billeder\\Pictures'
country = 'Denmark' # Change this manually

# Iterate over files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
        file_path = os.path.join(folder_path, filename)

        # Get the last change date using Windows file attributes
        last_change_timestamp = os.path.getmtime(file_path)
        last_change_date = datetime.datetime.fromtimestamp(last_change_timestamp)

        # Read the image file as binary data
        with open(file_path, 'rb') as image_file:
            image_data = image_file.read()

        # Insert the data into the database
        with connection.cursor() as cursor:
            insert_query = sql.SQL('INSERT INTO pi_data.pictures ("file_name", "content", "country", "photo_taken") VALUES (%s, %s, %s, %s)')
            cursor.execute(insert_query, (filename, psycopg2.Binary(image_data), country, last_change_date))

# Commit the transactions and close the connection
connection.commit()
connection.close()